<a href="https://colab.research.google.com/github/RuiqiTang/Learning-Diffusion/blob/main/ddim_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/RuiqiTang/Learning-Diffusion.git

Cloning into 'Learning-Diffusion'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 30 (delta 5), reused 25 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 11.99 KiB | 11.99 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [8]:
cd Learning-Diffusion

/content/Learning-Diffusion


In [15]:
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb
from torchvision import datasets,transforms

from models.UNet import UNetModel
from DDIM.ddim import GaussianDiffusion
from Utils.plotting import plot_image_grid

In [16]:
batch_size=64
timesteps=500
epochs=10
device='cuda' if torch.cuda.is_available() else 'cpu'


transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5],std=[0.5])
])

In [17]:
dataset=datasets.MNIST('/Data',train=True,download=True,transform=transform)
train_loader=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)

In [18]:
# Load: Model
model=UNetModel(
    in_channels=1,
    model_channels=96,
    out_channels=1,
    channel_mult=(1,2,2),
    attention_resolution=[]
)
model.to(device)

gaussian_diffusion=GaussianDiffusion(timesteps=timesteps)
optimizer=torch.optim.Adam(model.parameters(),lr=5e-4)

In [19]:
images:torch.Tensor
epoch_loss:float

wandb.init(
    project='ddim-mnist',
    config={
        "batch_size":batch_size,
        "timesteps":timesteps,
        "epochs":epochs,
        "lr":5e-4,
        "model_channels":96,
    }
)
wandb.watch(model,log='all')

for epoch in tqdm(range(epochs)):
    epoch_loss=0.0
    for step,(images,labels) in enumerate(train_loader):
        optimizer.zero_grad()
        batch_size=images.shape[0]
        images=images.to(device)

        '''
            为每个时间步均匀地采样一个时间步
            每个样本需要在不同的时间步上进行加噪或去噪，通过均匀采样t可以让模型学习到所有时间步的特征，确保对不同程度的噪声都有泛化能力
        '''
        t=torch.randint(0,timesteps,(batch_size,),device=device).long()
        loss=gaussian_diffusion.train_losses(model,images,t)

        loss.backward()
        optimizer.step()

        epoch_loss+=loss.item()
        global_step=epoch*len(train_loader)+step
        wandb.log({
            'train/loss':loss.item(),
            'epoch':epoch,
            'step':global_step
        })

    avg_loss=epoch_loss/len(train_loader)
    wandb.log({'train/epoch_loss':avg_loss,'epoch':epoch})

  0%|          | 0/10 [00:00<?, ?it/s]


TypeError: log(): argument 'input' (position 1) must be Tensor, not int